In [ ]:
import subprocess
import os
import csv
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [ ]:
# function to print elapsed time
def calculate_elapsed_time(start_time, end_time):
    elapsed_time = end_time - start_time
    total_seconds = elapsed_time.total_seconds()
    seconds = int(total_seconds % 60)
    total_minutes = total_seconds // 60
    minutes = int(total_minutes % 60)
    hours = int(total_minutes // 60)
    
    seconds = str(seconds).zfill(2)
    minutes = str(minutes).zfill(2)
    hours = str(hours).zfill(2)

    return str(hours) + ':' + str(minutes) + ':' + str(seconds)

# Initialise

### Read parameter sets to simulate

In [ ]:
# read parameters to run
table = pd.read_csv('data/parametersets_to_run.csv')
table

### Initialise empty the output dataframe

In [ ]:
# Initialise dataframe for mode choice distributions (1 line = 1 distribution = 1 iteration)
df = pd.DataFrame(columns=["CAR_DRIVER", "CAR_PASSENGER", "BIKE", "BUS_TRAM", "TRAIN", "WALK"])

for i in range(0, len(table)):
    df.to_csv("output/iterations/output_proportions_" + str(i) + ".csv", index=False)

# Call Sim2APL iteratively

### Prepare the Java command

In [ ]:
# baseline of the Java command
config = "--config src/main/resources/config_DHZW_full.toml"
modeliberal = "--mode-liberal 0.5"
modeconservative = "--mode-conservative 0.5"
output = "-o output/deskrun"
parametersPath = "--parameter_file ../data/parametersets_to_run.csv"

base_cmd = "java -cp target/sim2apl-dhzw-simulation-1.0-SNAPSHOT-jar-with-dependencies.jar main.java.nl.uu.iss.ga.Simulation" + " " + config + " " + modeliberal + " " + modeconservative + " " + output + " " + parametersPath

# set current directory the folder of Sim2APL so I can execute the jar with the correct paths
if(os.path.basename(os.getcwd()) != 'DHZW-simulation_Sim-2APL'):
    new_directory = os.path.join(os.getcwd(), 'DHZW-simulation_Sim-2APL')
    new_directory = new_directory.replace('\\', '/')
    os.chdir(new_directory)

### Main loop to call the simulations

In [ ]:
idx_restart = 0 # in case somethings breaks it is easy to continue

In [ ]:
time_beginning_all = datetime.now()

for idx, row in tqdm(table.iterrows(), total=table.shape[0]):
    if idx >= idx_restart:
        # repeat each parameter set 5 times
        for iteration in range(0, 5):

            # output file for this iteration
            outputPath = f'--output_file=../output/iterations/output_proportions_{idx}.csv'
            cmd = base_cmd + ' ' + outputPath
            
            # parameterset to use
            arg = f'--parameterset_index={idx+1}'
            full_command = f'{cmd} {arg}'
            
            time_beginning_iteration = datetime.now()
            
            print('parameter number: ' + str(idx) + ' - iteration: ' + str(iteration) + ' START || Time now: ' + time_beginning_iteration.strftime("%H:%M:%S"))
                                    
            # Capture the output of the Java program
            try:
                output = subprocess.check_output(full_command, stderr=subprocess.STDOUT, universal_newlines=True)
            except subprocess.CalledProcessError as e:
                print(f"Java program exited with non-zero return code: {e.returncode}")
                print(f"Error message: {e.output}")
                exit(1)
                
            time_end_iteration = datetime.now()
         
            print('parameter number: ' + str(idx) + ' - iteration: ' + str(iteration) + ' END   || Time now: ' + time_end_iteration.strftime("%H:%M:%S") + ' | Duration iteration: ' + calculate_elapsed_time(time_beginning_iteration, time_end_iteration) + ' | Elapsed from start: ' + calculate_elapsed_time(time_beginning_all, time_end_iteration))